In [1]:
import pandas as pd
import numpy as np
import math
from pathlib import Path

# --------- INPUT DATA FILE ----------
file_path = Path(r"D:\climate change\Tutorial_Climate Data_2025-26.xlsx")

# --------- OUTPUT RESULT FILE -------
output_file = Path(r"D:\climate change\VNR_Test_Results.xlsx")

try:
    xls = pd.ExcelFile(file_path)
    print(f"✅ Successfully loaded {file_path}")
except FileNotFoundError:
    print(f"❌ ERROR: File not found at {file_path}")
    exit()
except Exception as e:
    print(f"❌ ERROR: Could not load Excel file. {e}")
    exit()

results = []

print("Running Homogeneity Tests (Von Neumann, Cumulative Deviation)...")

for sheet in xls.sheet_names:
    print(f"\n📄 Processing Sheet: {sheet}")
    try:
        df = pd.read_excel(file_path, sheet_name=sheet)
    except Exception as e:
        print(f"  ...Could not read sheet '{sheet}'. Error: {e}")
        continue
        
    num_df = df.select_dtypes(include=[np.number])
    num_df = num_df.drop(columns=["Lon", "Lat"], errors="ignore")

    if num_df.empty:
        print("  ...No numeric data columns found.")
        continue

    for col in num_df.columns:
        x = num_df[col].dropna().values
        n = len(x)

        if n < 5:
            # Append NaNs for numeric columns to match data structure
            results.append([sheet, col, np.nan, "Insufficient data", np.nan, np.nan])
            continue
            
        mean_x = np.mean(x)
            
        # ---------- Von Neumann Ratio Test ----------
        numerator = np.sum((x[:-1] - x[1:])**2)
        # Use sum of squares for denominator
        denominator = np.sum((x - mean_x)**2) 

        if denominator == 0:
            # Data is constant, not random
            results.append([sheet, col, np.nan, "Constant Data", 0.0, 0.0])
            continue

        VN = numerator / denominator
        vn_result = "Random (Homogeneous)" if VN >= 2 else "Non-Random / Inhomogeneous"

        # ---------- Cumulative Deviation Test ----------
        sd_x = np.std(x, ddof=1) # Sample std dev
        
        # sd_x check is redundant if denominator check passed, but good for safety
        if sd_x == 0:
            results.append([sheet, col, round(VN,2), vn_result, 0.0, 0.0])
            continue

        Sk = np.cumsum((x - mean_x) / sd_x)

        Q = np.max(np.abs(Sk))
        R = np.max(Sk) - np.min(Sk)

        # Normalize (for table comparison)
        Q_norm = Q / np.sqrt(n)
        R_norm = R / np.sqrt(n)

        # Append with 2 decimal rounding
        results.append([
            sheet, col, 
            round(VN, 2), vn_result, 
            round(Q_norm, 2), round(R_norm, 2)
        ])
    
    print(f"  ...processed {len(num_df.columns)} stations.")

# Convert to DataFrame
try:
    result_df = pd.DataFrame(results, columns=[
        "Sheet", "Variable", "Von Neumann N", "VN Interpretation",
        "Q/sqrt(n)", "R/sqrt(n)"
    ])

    # Save to Excel
    result_df.to_excel(output_file, index=False)
    print("\n✅ Homogeneity Tests Completed")
    print(f"📁 Results saved to: {output_file}")

except Exception as e:
    print(f"\n❌ ERROR: Could not save results to {output_file}. {e}")


✅ Successfully loaded D:\climate change\Tutorial_Climate Data_2025-26.xlsx
Running Homogeneity Tests (Von Neumann, Cumulative Deviation)...

📄 Processing Sheet: Rainfall
  ...processed 23 stations.

📄 Processing Sheet: Tmax
  ...processed 23 stations.

📄 Processing Sheet: Tmin
  ...processed 23 stations.

✅ Homogeneity Tests Completed
📁 Results saved to: D:\climate change\VNR_Test_Results.xlsx
